In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np
import random
import pandas as pd
pd.set_option('display.max_columns', None)

import torch
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F

import neptune
from neptune_pytorch import NeptuneLogger
from neptune.utils import stringify_unsupported

import datetime
import pytz
timeZone = pytz.timezone('America/Los_Angeles')
from operator import itemgetter

from eegUtils import *
import torchModels
from myUtils import *

In [ ]:
import importlib
importlib.reload(torchModels)

# Read the data from disk

In [2]:
dataMultiply = 10**5
sFreq = 256
arrayInCompressedFile = 'arr_7'

data = np.load('/blue/gkalamangalam/jmark.ettinger/eegCompress/processedData/elimPeaksSVD001.npz')[arrayInCompressedFile]
nChannel, nSample = data.shape
data = (data * dataMultiply).astype('float32')
dataTensor = torch.tensor(data)
if torch.cuda.is_available():
    dataTensor = dataTensor.to('cuda')
    print("Data has been moved to GPU")
else:
    print("Data is on CPU")
print(data.shape)

Data has been moved to GPU
(19, 1100367)


In [3]:
path = '/blue/gkalamangalam/jmark.ettinger/eegCompress/processedData/kmeansModels/kmeansModel_001_block7_1stack.npz'
npzfile = np.load(path)
centroids = npzfile['arr_0']

# Define Model, Optimizer, DataSet and Optionally Load All

In [4]:
# Set parameters
modelType = 'kmeans'
loadBool = 0
#modelPath = '/content/drive/MyDrive/NeuroResearch/Data/eegCompress/models/savedModel_03-28 19:04_-0.034.pt'
numSampleInput = 20
numSampleOutput = 1


initDict = {'kmeansInit': centroids, 'dataTensor': dataTensor, 'numSampleInput':numSampleInput}

model, dataset, loss_function = torchModels.makeModel(modelType, initDict)
# modelType, nChannel, numSampleInput, numSampleOutput, dataTensor

sizeOfModel = modelSize(model)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.0)
#optimizer = torch.optim.Adam(model.parameters())#, lr = 1e-1, weight_decay = 1e-8)

if loadBool:
    model, optimizer, totalEpoch, loss = loadModel(modelPath, model, optimizer, trainBool=True)
    print("Model has been loaded: " + modelPath)
else:
    totalEpoch = 0

print(model)

if torch.cuda.is_available():
    model.to('cuda')
    print("Model has been moved to GPU")
else:
    print("Model is on CPU")


model size: 9.619MB
conv1dKmeans(
  (myNet): Sequential(
    (0): Conv1d(19, 50, kernel_size=(3,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(50, 50, kernel_size=(3,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(50, 50, kernel_size=(3,), stride=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=700, out_features=19, bias=True)
  )
)
Model has been moved to GPU


In [5]:
logFlag = True

if logFlag:
  run = neptune.init_run(
      project="jettinger35/eegCompress",
      api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzMjFlMzY2MS1iOWZiLTRmZWEtOGMwNy0zOTVkMTljOGVjYTMifQ==",
      #with_id="EEG-116"
      )

  npt_logger = NeptuneLogger(
      run=run,
      model=model)

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/jettinger35/eegCompress/e/EEG-237


# Training loop

In [ ]:
epochs = 5
batchSize = 32
numRandomPlot = 2
secondsToPlot = 5
saveEveryNEpochs = 0 # 0 for no saving

# initialize
breakFlag = False
loader = torch.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=False, sampler=None)

# training loop
for epoch in range(epochs):
    counter = 0

    myPrint("Epoch: " + str(totalEpoch))
    if breakFlag:
        myPrint("Break!")
        break
    for (thisBlock, label) in loader:
        counter += 1
        prediction = model(thisBlock)
        if np.any(np.isnan(prediction.detach().cpu().numpy())):
            myPrint("NaN detected.  Counter: " + str(counter))
            breakFlag = True
            break
        loss = loss_function(prediction, label).mean()

        optimizer.zero_grad()
        loss.backward()

        max_norm = 1.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        optimizer.step()
        if logFlag:
            run[npt_logger.base_namespace]["train/log_loss"].append(np.log(loss.item()))

    if logFlag:
      # plot random locations for original and predicted for comparison
      for i in range(numRandomPlot):
            startPlot = random.randint(0, nSample - (secondsToPlot * sFreq))
            fig, original, predicted = timeSeriesCompare(model, startPlot, secondsToPlot, sFreq, data, numSampleInput)
            plt.title("Epoch, Start, Blocks: " + str((totalEpoch, startPlot, secondsToPlot)))
            run["fig"].append(fig)
            plt.close()

    totalEpoch += 1

    print("Predicting and generating sampler...")
    sampler, predicted, residualMeasure = samplerMake(model, numSampleInput, data)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=False, sampler=sampler)

    if saveEveryNEpochs > 0 and (epoch + 1) % saveEveryNEpochs == 0:
        saveModel(model, optimizer, totalEpoch, loss, predicted)

myPrint("Finished training...")

04-15 11:33: Epoch: 0


# Save the model

In [ ]:
sampler, predicted, residualMeasure = samplerMake(model, numSampleInput, data)
saveModel(model, optimizer, totalEpoch, loss, predicted)

# Compare the time series

In [ ]:
plotBool = 1

if plotBool:
  startPlot = 1000
  secondsToPlot = 5
  channel = 0
  plotOption = 'both'

  fig, original, predicted64 = timeSeriesCompare(model, startPlot, secondsToPlot, sFreq, data, numSampleInput, channel, plotOption)
  plt.show(fig)

# Save original and predicted data for local graphical comparison

In [ ]:
predicted = predictEEG(model, None, data)

In [ ]:
path = '/content/drive/MyDrive/NeuroResearch/Data/eegCompress/processedData/origAndPredictedSVD001_block7.npz'
dataToSaveList = [data, predicted]
np.savez_compressed(path, *dataToSaveList)

# Show network parameters

In [ ]:
for param in model.parameters():
    print((param.shape, param.device))

# Scratch

In [ ]:
# EXPORT ONNX FOR VISUALIZATION IN NETRON APP

visualizationPath = '/content/drive/MyDrive/NeuroResearch/Data/eegCompress/models/model.onnx'
dataset = datasetMake(torch.tensor(data[:,100:200]), model.numSampleInput, model.typeCode)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None)
batch, label = next(loader.__iter__())
yhat = model(batch)

torch.onnx.export(model, batch, f=visualizationPath)